##  Importação de Bibliotecas

In [47]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import gridspec
from matplotlib import pyplot as plt
from matplotlib.ticker import StrMethodFormatter, NullFormatter
import plotly.express as px
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import mysql.connector as connection

## Extração de Dados

In [48]:
#---------------------------------------------
#-----(Carregando dados do banco de dados)----
#---------------------------------------------

try:
    mydb = connection.connect(host="localhost", database = 'housedb',user="root", passwd="Master2010",use_pure=True) #Conexão com o BD
    query = "Select * from houses;" #Query da tabela house
    data = pd.read_sql(query,mydb) #Salvando resultado da Query em um pandas dataframe
    mydb.close() #fechando conexão com o BD
except Exception as e:
    mydb.close()
    print(str(e))

/Users/luizramos/.pyenv/versions/LabProdSoftware/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [49]:
# ----(Carrehando Token Mapbox)----

token = open(r'MapboxToken.txt').read()

## Transformação de Dados

In [50]:
#---------------------------------------------------
#----(Retira notação científica Pandas e Numpy)-----
#---------------------------------------------------

np.set_printoptions(suppress=True)
pd.set_option('display.float_format', '{:.2f}'.format)

In [51]:
data['dates'] = pd.to_datetime( data['dates'], format='%Y-%m-%d' )
data.dtypes

id                        int64
dates            datetime64[ns]
price                   float64
bedrooms                  int64
bathrooms               float64
sqft_living               int64
sqft_lot                  int64
floors                  float64
waterfront                int64
views                     int64
conditions                int64
grade                     int64
sqft_above                int64
sqft_basement             int64
yr_built                  int64
yr_renovated              int64
zipcode                   int64
lat                     float64
lon                     float64
sqft_living15             int64
sqft_lot15                int64
dtype: object

In [52]:
pd.to_datetime(data['dates'].min())

Timestamp('2014-05-02 00:00:00')

In [53]:
#--------------------------------------------------------------------------------------------------
# ----------------(Transformação de dados / Criação de novas tabelas)------------------------------
#--------------------------------------------------------------------------------------------------

#Editando tabela dates - para para datetime e no formato Ano, Mês e dia
data['dates'] = pd.to_datetime( data['dates'] ).dt.strftime( '%Y-%m-%d' )

#Criando tabela "year" em datetime e no formado Ano
data['year'] = pd.to_datetime( data['dates'] ).dt.strftime( '%Y' )

#Criando tabela "year_week" em datetime e no formado Ano-Mês
data['year_week'] = pd.to_datetime( data['dates'] ).dt.strftime( '%Y-%U' )

#Criando tabela "is_waterfront"
data['is_waterfront'] = data['waterfront'].apply( lambda x: 'sim' if x == 1 else 'não' )

In [54]:
#--------------------------------------
# ----(Botões Iterativos - Mapa)-------
#--------------------------------------

#Filtro - Preço Máximo
price_limit = widgets.IntSlider(
    value = data['price'].mean(),
    min = data['price'].min(),
    max = data['price'].max(),
    step = 1,
    description='Preço Maximo',
    disable=False,
    style={'description_width': 'initial'}
)

#Filtro - Vista para o mar
waterfront_bar = widgets.Dropdown(
    options = data['is_waterfront'].unique().tolist(),
    value = 'sim',
    description = 'Vista para o mar',
    disable=False
)

#Filtro - Tamanho mínimo da sala de estar
livingroom_limit = widgets.IntSlider(
    value = int( data['sqft_living'].mean() ), 
    min = data['sqft_living'].min(),
    max = data['sqft_living'].max(),
    step = 1,
    description='Tamanho mínimo da sala de estar',
    disable=False,
    style={'description_width': 'initial'}
)

#Filtro - Numéro mínimo de banheiros
bathroom_limit = widgets.IntSlider(
    value = int( data['bathrooms'].mean() ), 
    min = data['bathrooms'].min(),
    max = data['bathrooms'].max(),
    step = 1,
    description='Numéro mínimo de banheiros',
    disable=False,
    style={'description_width': 'initial'}
)

#Filtro - Tamnho mínimo de sótão
basement_limit = widgets.IntSlider(
    value = int( data['sqft_basement'].mean() ), 
    min = data['sqft_basement'].min(),
    max = data['sqft_basement'].max(),
    step = 1,
    description='Tamnho mínimo de sótão',
    disable=False,
    style={'description_width': 'initial'}
)

#Filtro - Condição Mínima
condition_limit = widgets.IntSlider(
    value = int( data['conditions'].mean() ), 
    min = data['conditions'].min(),
    max = data['conditions'].max(),
    step = 1,
    description='Condição Mínima',
    disable=False,
    style={'description_width': 'initial'}
)

#Filtro - Ano de construção
year_limit = widgets.IntSlider(
    value = int( data['yr_built'].mean() ), 
    min = data['yr_built'].min(),
    max = data['yr_built'].max(),
    step = 1,
    description='Ano de construção',
    disable=False,
    style={'description_width': 'initial'}
)

In [55]:
data.dtypes

id                 int64
dates             object
price            float64
bedrooms           int64
bathrooms        float64
sqft_living        int64
sqft_lot           int64
floors           float64
waterfront         int64
views              int64
conditions         int64
grade              int64
sqft_above         int64
sqft_basement      int64
yr_built           int64
yr_renovated       int64
zipcode            int64
lat              float64
lon              float64
sqft_living15      int64
sqft_lot15         int64
year              object
year_week         object
is_waterfront     object
dtype: object

In [56]:
#------------------------------------------
# ----(Botões Iterativos - Gráficos)-------
#------------------------------------------

# Filtro - Data máxima disponível
date_limit = widgets.SelectionSlider( 
    options = data['dates'].sort_values().unique().tolist(),
    value='2014-12-01',
    description='Data máxima disponível',
    disable=False,
    continuous_update=False,
    orientation='horizontal',
    style={'description_width': 'initial'},
    readout=True
)

# Filtro - Ano máximo de reforma
year_limit = widgets.SelectionSlider( 
    options = data['yr_renovated'].sort_values().unique().tolist(),
    value=2000,
    description='Ano máximo reforma',
    disable=False,
    continuous_update=False,
    orientation='horizontal',
    style={'description_width': 'initial'},
    readout=True
)

# Filtro - Vista para o Mar 
waterfront_limit = widgets.Checkbox(
    value=False,
    description='Vista para o mar?',
    disabled=False,
    indent=False
)

## Carregando Dados Mapas

In [57]:
#---------------------------------------------------------------------------------------
#---------------------------(Carregando função loadmap)---------------------------------
#---------------------------------------------------------------------------------------

def loadmap(data, waterfront, limit, livingroom_limit, bathroom_limit, basement_limit, condition_limit, year_limit ):

#----------(Filtrando data)----------------

    houses = data[(data['price'] <= limit) & 
                (data['is_waterfront'] == waterfront) &
                (data['sqft_living'] >= livingroom_limit) &
                (data['bathrooms'] >= bathroom_limit) &
                (data['sqft_basement'] >= basement_limit) &
                (data['conditions'] >= condition_limit) &
                (data['yr_built'] >= year_limit )][['id', 'lat', 'lon', 'price']].copy()

#-----------(Carregando mapa)---------------

    fig = px.scatter_mapbox( houses, 
                             lat="lat", 
                             lon="lon", 
                             size="price",
                             color_discrete_sequence=['Gainsboro'],
                             size_max=15, 
                             height=300,
                             zoom=10)

    fig.update_layout(mapbox_style='dark', mapbox_accesstoken=token)
    fig.update_layout(height=600, margin={"r":0,"t":0,"l":0,"b":0})
    fig.show()


In [58]:
#------(Carregando mapa com os filtros)-----------

widgets.interactive( loadmap, 
                     data=fixed(data), 
                     waterfront=waterfront_bar, 
                     limit=price_limit, 
                     livingroom_limit=livingroom_limit,
                     bathroom_limit=bathroom_limit,
                     basement_limit=basement_limit,
                     condition_limit=condition_limit,
                     year_limit=year_limit
                    )

interactive(children=(Dropdown(description='Vista para o mar', index=1, options=('não', 'sim'), value='sim'), …

## Carregando Dados Gráficos

In [61]:

def update_graph( data, date_limit, year_limit, waterfront_limit ):
    
#  ----(Filtrando data)------

    df = data[(data['dates'] <= date_limit) & 
              (data['yr_renovated'] >= year_limit) &
              (data['waterfront'] == waterfront_limit)].copy()

    fig = plt.figure( figsize=(24, 12) )
    specs = gridspec.GridSpec( ncols=2, nrows=2, figure=fig )

    ax1 = fig.add_subplot( specs[0, 0] ) # First Row
    ax2 = fig.add_subplot( specs[0, 1] ) # First Row First Column
    ax3 = fig.add_subplot( specs[1, :] ) # Second Row First Column

    
#   ----(Gráfico 1 - Preço médio por ano)----
    
    
    #Criando Gráfico 1
    by_year = df[['id', 'year']].groupby( 'year' ).sum().reset_index()
    ax1.bar( by_year['year'], by_year['id'] )
    ax1.set_title( "Preço médio por ano" )
    ax1.tick_params(axis='x', rotation=0)
    
    # Retirar notação científica - Gráfico 1
    ax1.yaxis.set_major_formatter(StrMethodFormatter('{x:.0f}'))
    ax1.yaxis.set_minor_formatter(NullFormatter())
    
    
#   ----(Gráfico 2 - Preço médio por semana)----
    
     #Criando Gráfico 2
    df['year_week'] = pd.to_datetime( df['dates'] ).dt.strftime( '%Y-%U' )
    by_week_of_year = df[['id', 'year_week']].groupby( 'year_week' ).mean().reset_index()
    ax2.bar( by_week_of_year['year_week'], by_week_of_year['id'] )
    ax2.set_title( "Preço médio por semana" )
    ax2.tick_params(axis='x', rotation=60)
    
    # Retira notação científica -  Gráfico 2
    ax2.yaxis.set_major_formatter(StrMethodFormatter('{x:.0f}'))
    ax2.yaxis.set_minor_formatter(NullFormatter())
    

#   ----(Gráfico 3 - Preço médio por dia)----

    # Criando Gráfico 3 - Preço médio por dia
    by_day = df[['id', 'dates']].groupby( 'dates' ).mean().reset_index()
    ax3.plot( by_day['dates'], by_day['id'] )
    ax3.set_title( "Preço médio por dia" )
    ax3.tick_params(axis='x', rotation=90)
    
    # Retira notação científica - Gráfico 3
    ax3.yaxis.set_major_formatter(StrMethodFormatter('{x:.0f}'))
    ax3.yaxis.set_minor_formatter(NullFormatter())
    
    

In [62]:
#-----------(Carregando gráficos com filtros)----------------

widgets.interactive( update_graph, 
                     data = fixed( data ), 
                     date_limit=date_limit,
                     year_limit=year_limit,
                     waterfront_limit=waterfront_limit )

interactive(children=(SelectionSlider(continuous_update=False, description='Data máxima disponível', index=48,…